In [1]:
import polars as pl
import numpy as np

ModuleNotFoundError: No module named 'polars'

In [14]:
df = pl.read_csv('../monitoring_data/trading_history.csv')
# BEGIN: Convert 'trade_date' column to date type
df = df.with_columns(pl.col('trade_date').str.strptime(pl.Date, '%Y/%m/%d'))
# END:
df.head()

trade_date,ISIN_code,trade_quantity,trade_price,settlement_amount,sell_is_one
date,str,i64,i64,i64,i64
2023-04-04,"""JP90C000GKC6""",5159,19383,10000,0
2023-04-04,"""JP90C000H1T1""",5812,17205,10000,0
2023-05-02,"""JP90C000GKC6""",4915,20349,10000,0
2023-05-08,"""JP90C000GKC6""",101,19826,200,0
2023-05-08,"""JP90C000H1T1""",5665,17652,10000,0


In [46]:
# 元本計算のため売却ならマイナスをつける
df = df.with_columns(
    pl.when(pl.col('sell_is_one') == 0)
    .then(pl.col('trade_quantity') * pl.col('trade_price') / 10000)
    .otherwise(-(pl.col('trade_quantity') * pl.col('trade_price') / 10000))
    .alias('trade_value')
)

# 保有株数の計算のため，売却ならマイナスをつける
df = df.with_columns(
    pl.when(pl.col('sell_is_one') == 0)
    .then(pl.col('trade_quantity'))
    .otherwise(-pl.col('trade_quantity'))
    .alias('trade_quantity_held')
)

principal = round(df.select(pl.sum('trade_value')).item()) # 元本
number_of_units_held = df.select(pl.sum('trade_quantity_held')).item() # 保有株数
trade_price_net = round(principal / number_of_units_held * 10000) # 取引価格の平均

print(f"principal: {principal}")
print(f"number of units held: {number_of_units_held}")
print(f"trade price net: {trade_price_net}")

principal: 873421
number of units held: 379591
trade price net: 23010
